## Install the Roboflow and ultralytics liabrary

In [1]:
!pip install roboflow -q
!pip install ultralytics -q

## Import the libraries

In [4]:
from roboflow import Roboflow
from ultralytics import YOLO
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("ROBOFLOW_API_KEY")


## Download the dataset

In [7]:
rf = Roboflow(api_key=secret_value_0)
project = rf.workspace("material-identification").project("garbage-classification-3")
version = project.version(1)
dataset = version.download("yolov11")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to GARBAGE-CLASSIFICATION-3-1 in yolov11:: 100%|██████████| 34742/34742 [00:03<00:00, 9575.25it/s] 


## Dataset info.

In [8]:
print(dataset.name)
print(dataset.version)
print(dataset.model_format)
print(dataset.location)

GARBAGE CLASSIFICATION 3
1
yolov11
/kaggle/working/GARBAGE-CLASSIFICATION-3-1


## Train, Valid, Test. Paths

In [9]:
TRAIN_IMAGES_PATH = "/kaggle/working/GARBAGE-CLASSIFICATION-3-1/train/images"
VALID_IMAGES_PATH = "/kaggle/working/GARBAGE-CLASSIFICATION-3-1/valid/images"
TEST_IMAGES_PATH = "/kaggle/working/GARBAGE-CLASSIFICATION-3-1/test/images"

In [11]:
print(f"Total Train Images: {len(os.listdir(TRAIN_IMAGES_PATH))}")
print(f"Total Validation Images: {len(os.listdir(VALID_IMAGES_PATH))}")
print(f"Total Test Images: {len(os.listdir(TEST_IMAGES_PATH))}")

Total Train Images: 14225
Total Validation Images: 2098
Total Test Images: 1042


## GPU availability

In [12]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Dec  6 08:40:27 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## upload the YOLOv11n model for traning.

In [13]:
model = YOLO("yolo11n.pt")

100%|██████████| 5.35M/5.35M [00:00<00:00, 75.7MB/s]


In [14]:
model.info()

YOLO11n summary: 319 layers, 2,624,080 parameters, 0 gradients, 6.6 GFLOPs


(319, 2624080, 0, 6.614336)

## Hyperparameters, finetuning.

In [15]:
IMG_SIZE = 640
BATCH_SIZE = -1
EPOCHS = 100

In [17]:
results = model.train(
    data = '/kaggle/input/data-1/data.yaml',
    task = 'detect',
    imgsz = IMG_SIZE,
    epochs = EPOCHS,
    batch = -1,
    device = 0,
    verbose = True,
    seed = 93,
    plots = True,
    val = True,
    name = 'Garbage_Classifier',
    project = 'Garbage_Classifier_Project',
    patience = 10,
    save = True,
    optimizer = 'SGD',
    lr0 = 0.001,
    lrf = 0.0001,
    momentum = 0.937,
    weight_decay = 0.005,
    cos_lr=True,
    profile=True,
    freeze=3,
    pretrained=True,
    dropout=0.2
)

Ultralytics 8.3.43 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/kaggle/input/data-1/data.yaml, epochs=100, time=None, patience=10, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=Garbage_Classifier_Project, name=Garbage_Classifier2, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=93, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=True, freeze=3, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.2, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, 

100%|██████████| 755k/755k [00:00<00:00, 17.6MB/s]


Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /kaggle/working/GARBAGE-CLASSIFICATION-3-1/train/labels... 14225 images, 0 backgrounds, 0 corrupt: 100%|██████████| 14225/14225 [00:12<00:00, 1102.90it/s]


train: New cache created: /kaggle/working/GARBAGE-CLASSIFICATION-3-1/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (Tesla T4) 14.74G total, 0.10G reserved, 0.06G allocated, 14.58G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
     2591010       6.446         0.247         54.71         159.1        (1, 3, 640, 640)                    list
     2591010       12.89         0.380         30.49         57.86        (2, 3, 640, 640)                    list
     2591010       25.78         0.661         30.83         51.21        (4, 3, 640, 640)                    list
     2591010       51.57         1.267         31.53         5

...

     78/100      14.9G      1.046     0.8174      1.201        138        640: 100%|██████████| 246/246 [02:15<00:00,  1.81it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:16<00:00,  1.14it/s]

                   all       2098      18916      0.663      0.509      0.582      0.406


EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 68, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

78 epochs completed in 3.385 hours.
Optimizer stripped from Garbage_Classifier_Project/Garbage_Classifier2/weights/last.pt, 5.5MB
Optimizer stripped from Garbage_Classifier_Project/Garbage_Classifier2/weights/best.pt, 5.5MB

Validating Garbage_Classifier_Project/Garbage_Classifier2/weights/best.pt...
Ultralytics 8.3.43 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 238 layers, 2,583,322 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:18<00:00,  1.04it/s]


                   all       2098      18916      0.642      0.517      0.581      0.406
         BIODEGRADABLE        676      13637      0.819      0.516      0.663      0.385
             CARDBOARD        438       1292      0.773      0.556      0.665      0.505
                 GLASS        805       2380      0.879       0.74      0.839       0.63
                 METAL        398       1360      0.792      0.637      0.736      0.516
                 PAPER         15         33      0.122      0.152      0.101     0.0773
               PLASTIC         91        214      0.467      0.504      0.483      0.325
Speed: 0.2ms preprocess, 2.7ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to Garbage_Classifier_Project/Garbage_Classifier2


## Validation

In [18]:
evaluation = model.val(
    data = '/kaggle/input/data-1/data.yaml',
    task = 'detect',
    imgsz = 640,
    iou = 0.6,
    batch = 32,
    save_json = True,
    save_txt = True,
    device = 0,
    verbose = True,
    seed = 93,
    plots = True,
    name = 'Model-detect-evaluation-1'
)

Ultralytics 8.3.43 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 238 layers, 2,583,322 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning /kaggle/working/GARBAGE-CLASSIFICATION-3-1/valid/labels.cache... 2098 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2098/2098 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 66/66 [02:49<00:00,  2.57s/it]


                   all       2098      18916      0.642      0.532       0.59      0.407
         BIODEGRADABLE        676      13637      0.811      0.535      0.678      0.386
             CARDBOARD        438       1292       0.78      0.567      0.679      0.506
                 GLASS        805       2380      0.883      0.747      0.848       0.63
                 METAL        398       1360      0.787       0.65      0.747      0.517
                 PAPER         15         33      0.146      0.182      0.102     0.0772
               PLASTIC         91        214      0.446      0.512      0.487      0.326
Speed: 0.2ms preprocess, 4.2ms inference, 0.0ms loss, 1.2ms postprocess per image
Saving Garbage_Classifier_Project/Model-detect-evaluation-1/predictions.json...
Results saved to Garbage_Classifier_Project/Model-detect-evaluation-1


## predictions on the testset


In [19]:
predictions = model.predict(
    data = '/kaggle/input/data-1/data.yaml',
    source = '/kaggle/working/GARBAGE-CLASSIFICATION-3-1/test/images',
    task = 'detect',
    imgsz = 640,
    device = 0,
    verbose = True,
    seed = 93,
    plots = True,
    name = 'Model-segment-predictions-1',
    save_json = True,
)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1042 /kaggle/working/GARBAGE-CLASSIFICATION-3-1/test/images/metal1000_jpg.rf.f972491130c59d3bff7e97b22e8e2980.jpg: 640x640 1 METAL, 9.9ms
image 2/1042 /kaggle/working/GARBAGE-CLASSIFICATION-3-1/test/images/metal1005_jpg.rf.d736ab336e8fa92ce57727666e624259.jpg: 640x640 1 METAL, 8.5ms
image 3/1042 /kaggle/working/GARBAGE-CLASSIFICATION-3-1/test/images/metal100_jpg.rf.8605f3d87c15f6f37575fa2907d5bc6c.jpg: 640x640 2 METALs, 9.2ms
image 4/1042 /kagg

## quantize and export the model

In [20]:
model.export(format="onnx", int8=True, dynamic=True,half=True, data='/kaggle/input/data-1/data.yaml')

Ultralytics 8.3.43 🚀 Python-3.10.14 torch-2.4.0 CPU (Intel Xeon 2.00GHz)
WARNING ⚠️ half=True and int8=True are mutually exclusive, setting half=False.

PyTorch: starting from 'Garbage_Classifier_Project/Garbage_Classifier2/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 10, 8400) (5.2 MB)
requirements: Ultralytics requirements ['onnxslim', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.9/142.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.5/291.5 MB 298.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 260.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 281.3 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 16.2s, installed 2 packages: ['onnxslim', 'onnxruntime-gpu']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimmin

'Garbage_Classifier_Project/Garbage_Classifier2/weights/best.onnx'

## /kaggle/working/Garbage_Classifier_Project/Model-detect-evaluation-1download the results

In [21]:
!zip -r Classifier_Project.zip /kaggle/working/Garbage_Classifier_Project/Garbage_Classifier2

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.


  adding: kaggle/working/Garbage_Classifier_Project/Garbage_Classifier2/ (stored 0%)
  adding: kaggle/working/Garbage_Classifier_Project/Garbage_Classifier2/val_batch1_pred.jpg (deflated 6%)
  adding: kaggle/working/Garbage_Classifier_Project/Garbage_Classifier2/weights/ (stored 0%)
  adding: kaggle/working/Garbage_Classifier_Project/Garbage_Classifier2/weights/best.onnx (deflated 12%)
  adding: kaggle/working/Garbage_Classifier_Project/Garbage_Classifier2/weights/last.pt (deflated 10%)
  adding: kaggle/working/Garbage_Classifier_Project/Garbage_Classifier2/weights/best.pt (deflated 10%)
  adding: kaggle/working/Garbage_Classifier_Project/Garbage_Classifier2/val_batch1_labels.jpg (deflated 6%)
  adding: kaggle/working/Garbage_Classifier_Project/Garbage_Classifier2/confusion_matrix_normalized.png (deflated 20%)
  adding: kaggle/working/Garbage_Classifier_Project/Garbage_Classifier2/val_batch0_pred.jpg (deflated 13%)
  adding: kaggle/working/Garbage_Classifier_Project/Garbage_Classifier2

In [22]:
!zip -r Classifier_Project_eval.zip /kaggle/working/Garbage_Classifier_Project/Model-detect-evaluation-1

  adding: kaggle/working/Garbage_Classifier_Project/Model-detect-evaluation-1/ (stored 0%)
  adding: kaggle/working/Garbage_Classifier_Project/Model-detect-evaluation-1/val_batch1_pred.jpg (deflated 6%)
  adding: kaggle/working/Garbage_Classifier_Project/Model-detect-evaluation-1/val_batch1_labels.jpg (deflated 6%)
  adding: kaggle/working/Garbage_Classifier_Project/Model-detect-evaluation-1/confusion_matrix_normalized.png (deflated 20%)
  adding: kaggle/working/Garbage_Classifier_Project/Model-detect-evaluation-1/val_batch0_pred.jpg (deflated 13%)
  adding: kaggle/working/Garbage_Classifier_Project/Model-detect-evaluation-1/val_batch2_labels.jpg (deflated 7%)
  adding: kaggle/working/Garbage_Classifier_Project/Model-detect-evaluation-1/labels/ (stored 0%)
  adding: kaggle/working/Garbage_Classifier_Project/Model-detect-evaluation-1/labels/biodegradable1430_jpg.rf.71f1d8131db49e455a36161006752c8d.txt (deflated 65%)
  adding: kaggle/working/Garbage_Classifier_Project/Model-detect-evalua

In [23]:
!zip -r Classifier_Project_eval_last.zip/kaggle/working/Garbage_Classifier_Project/Garbage_Classifier


zip error: Nothing to do! (Classifier_Project_eval_last.zip/kaggle/working/Garbage_Classifier_Project/Garbage_Classifier.zip)
